<a href="https://colab.research.google.com/github/grayhacked/AI/blob/main/LAB_1_Data_manipulation_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB1. CNN manipulation

In [4]:
#Importation des bibliotheques
import torch  #Bibliotheques pour le deep learning
import torch.nn as nn  #Bibliotheques pour creer les reseaux de neurones
import torch.optim as optim  #Bibliotheques pour l'optimisation du modele
from torchvision import datasets, transforms  #Bibliotheques pour charger les donnees d'images comme MNIST
from torch.utils.data import DataLoader  #Bibliotheques pour creer des dataloaders,  Pour gérer les lots (batches) de données
#: DataLoader Permet de charger les données en petits morceaux appelés batches, pour les traiter plus facilement.


In [6]:
#Étape 2 : Chargement des Données

# Transformation des images (normalisation et conversion en tenseur)
transform = transforms.Compose([
    transforms.ToTensor(),# Convertir une image en tenseur (tableau numérique manipulable par PyTorch)
    transforms.Normalize((0.5,), (0.5,))  # Normalisation pour une moyenne de 0 et un écart-type de 1, Normaliser les valeurs des pixels (les ramener entre -1 et 1)
])

# Chargement des datasets MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True) #Ensemble d'entraînement (images utilisées pour entraîner le modèle).
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True) #Ensemble de test (images utilisées pour évaluer le modèle après l'entraînement).

# DataLoader pour itérer sur les datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
'''
DataLoader :

Coupe les données en petits groupes (batches) de 64 images pour un traitement plus rapide.
shuffle=True mélange les données à chaque époque pour que le modèle ne "triche" pas en apprenant l'ordre des images.
'''


In [7]:
#Étape 3 : Définition d’un CNN Simple
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # 1 canal (gris) -> 16 filtres
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Réduction de la taille
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # 16 -> 32 filtres
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # FC1 (entrée : 32 filtres réduits à 7x7)
        self.fc2 = nn.Linear(128, 10)  # FC2 (10 classes pour MNIST)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Convolution + ReLU + Pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Deuxième couche
        x = x.view(-1, 32 * 7 * 7)  # Aplatir les tenseurs
        x = torch.relu(self.fc1(x))  # Couche Fully Connected 1
        x = self.fc2(x)  # Couche Fully Connected 2
        return x

'''
Explications ligne par ligne :
Couches convolutives :

nn.Conv2d(1, 16, kernel_size=3) : La première couche prend une image avec 1 canal (niveau de gris) et produit 16 cartes de caractéristiques.
nn.Conv2d(16, 32, kernel_size=3) : La deuxième couche prend les 16 cartes produites et en crée 32.
ReLU :

nn.ReLU() : Applique la fonction d'activation ReLU après chaque convolution.

MaxPooling :
nn.MaxPool2d(kernel_size=2, stride=2) : Réduit la taille des cartes de caractéristiques de moitié.

Fully Connected Layers :
nn.Linear(32 * 7 * 7, 128) : Transforme les cartes de 32x7x7 en un vecteur de taille 128.
nn.Linear(128, 10) : Sortie de taille 10 (une pour chaque chiffre de 0 à 9).

Fonction forward :
Spécifie le chemin que les données suivent dans le réseau.
Applique ReLU après chaque convolution et utilise MaxPooling pour réduire la taille.
'''

In [8]:
#Étape 4 : Entraînement du Modèle
# Initialisation du modèle, de l’optimiseur et de la fonction de perte

model = SimpleCNN() # On crée une instance de notre modèle
criterion = nn.CrossEntropyLoss() # Fonction de perte pour la classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimiseur Adam pour ajuster les poids du modèle

# Boucle d'entraînement
for epoch in range(5):  # Entraîner pendant 5 époques
    model.train() # Mode entraînement
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad() # Réinitialise les gradients
        outputs = model(images)  # Passe avant (forward)
        loss = criterion(outputs, labels) # Calcule la perte
        loss.backward() # Calcul des gradients (backpropagation)
        optimizer.step() # Mise à jour des poids
        running_loss += loss.item()

    print(f"Époque {epoch+1}, Perte : {running_loss / len(train_loader):.4f}")


'''
Explications :
Critère : La fonction de perte mesure l’écart entre les prédictions et les vraies étiquettes.
Optimiseur : Ajuste les poids pour réduire la perte.
Boucle d’entraînement :
optimizer.zero_grad() : Réinitialise les gradients pour éviter leur accumulation.
loss.backward() : Propagation des erreurs pour ajuster les poids.
optimizer.step() : Mise à jour des poids.
'''

Époque 1, Perte : 0.1960
Époque 2, Perte : 0.0527
Époque 3, Perte : 0.0374
Époque 4, Perte : 0.0291
Époque 5, Perte : 0.0219


In [9]:
#Étape 5 : Évaluation du Modèle
# Test du modèle sur les données de test

model.eval() # # Mode évaluation (désactive certaines fonctions comme le dropout)
correct = 0
total = 0

with torch.no_grad():  # Pas de calcul de gradients ici
    for images, labels in test_loader:
        outputs = model(images)  # Prédictions
        _, predicted = torch.max(outputs.data, 1)  # Choix de la classe avec la probabilité la plus élevée
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Exactitude sur le dataset de test : {100 * correct / total:.2f}%")

'''
Explications :
Mode évaluation : Empêche le recalcul des gradients.
Prédictions :
torch.max(outputs.data, 1) : Retourne la classe ayant la probabilité maximale.
Calcul de l'exactitude :
Compare les prédictions aux vraies étiquettes.
'''

Exactitude sur le dataset de test : 98.93%
